### 8-puzzle

In this notebook, we will be writing the script that solves the 8 puzzle problem.

Before moving on, please review search.ipynb!

BUT before everything else....tell us who you are:

### Student Number = csd4300

### Name = Stivaktakis Giorgos

In [1]:
from search import *
import time
import random
import timeit

In [2]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board, where one of the
    squares is a blank. A state is represented as a tuple of length 9, where  element at
    index i represents the tile number  at index i (0 if it's an empty square) """
    
    max_actions = 10000000
    current_actions = 0
    
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)
        

    def find_blank_square(self, state):
        """Return the index of the blank square in a given state"""

        return state.index(0)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """

        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        index_blank_square = self.find_blank_square(state)
        
        if index_blank_square % 3 == 0:
            possible_actions.remove('LEFT')
        if index_blank_square < 3:
            possible_actions.remove('UP')
        if index_blank_square % 3 == 2:
            possible_actions.remove('RIGHT')
        if index_blank_square > 5:
            possible_actions.remove('DOWN')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        # blank is the index of the blank square
        blank = self.find_blank_square(state)
        new_state = list(state)

        delta = {'UP': -3, 'DOWN': 3, 'LEFT': -1, 'RIGHT': 1}
        neighbor = blank + delta[action]
        new_state[blank], new_state[neighbor] = new_state[neighbor], new_state[blank]
        self.current_actions +=1
        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        
        return (state == self.goal) or (self.current_actions>self.max_actions)

    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j] != 0:
                    inversion += 1

        return inversion % 2 == 0

    def h(self, node):
        """ Return the heuristic value for a given state. Default heuristic function used is 
        h(n) = number of misplaced tiles """

        return sum(s != g for (s, g) in zip(node.state, self.goal))


In [3]:
goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)
initial_state =tuple(random.sample(list(goal),9))
print(goal)
print(initial_state)

(1, 2, 3, 4, 5, 6, 7, 8, 0)
(7, 0, 4, 2, 1, 8, 6, 3, 5)


In [4]:
import time

def run_singleTest(initial_state):
    # your function needs to take as input the object of our problem
    # and return all the necessary information of the solution per algorithm for comparison
    puzzle = EightPuzzle(initial_state)
    results = []
    
    ########################################################
    #  add code here to test in a similar fashion as below #
    #  4  algorithms for the same initial_state            #
    ########################################################
    algorithms_to_test = [
       ('Astar search',astar_search),
       ('Breadth First Tree Search',breadth_first_tree_search),
         ('Depth First Tree Search',depth_first_tree_search),
         ('Depth Limited Search',depth_limited_search),

        # Add more algorithms here as needed
    ]
    
    for name,func in algorithms_to_test:
        # IMPORTANT must add this before each algorithm call to reset actions
        puzzle.current_actions = 0
        # start timing your execution
        start_time = time.time()
        
        # get final state
        final_state = func(puzzle)
        success = final_state.state == goal # check if final state is goal state
        print(final_state)
        # calculate time
        end_time = time.time() - start_time
        
        # save important parameters
        algorithm_results = {
            'Algorithm': name,
            'Initial State': initial_state,
            'Final_State': final_state,
            'Time': end_time,
            'Success': success,
        }
        # add algorithm results to returning parameter
        results.append(algorithm_results)


    return results


In [5]:
solvable = False
while not solvable:
    initial_state =tuple(random.sample(list(goal),9))
    solvable = EightPuzzle(initial_state).check_solvability(initial_state)

In [55]:
results = run_singleTest(initial_state)


print("Initial State: " + str(initial_state))
for result in results:
    final_state = list(result['Final_State'].state)
    if result['Final_State'] == goal:   
        print('Algorithm: ' + str(result['Algorithm']) + ' succeeded , Execution Time: ' + str(round(result['Time'],2)))
        print('Resulting solution: ' + str(result['Final_State'].solution()) + ' ending in board: ' + str(result['Final_State']))
    else:
        print('Algorithm: ' + str(result['Algorithm']) + ' failed , Execution Time: ' + str(round(result['Time'],2)))
        print('Resulting solution: ' + str(result['Final_State'].solution()) + ' ending in board: ' + str(result['Final_State']))


<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>
(1, 2, 3, 4, 5, 6, 7, 8, 0)
Initial State: (5, 4, 0, 1, 2, 8, 6, 7, 3)
Algorithm: Astar search failed , Execution Time: 7.4
Resulting solution: ['LEFT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'UP', 'RIGHT', 'DOWN', 'DOWN'] ending in board: <Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>


After filling in the function that conducts a comparison amongst algorithms for a single intial state.
Run 100 comparisons for random SOLVABLE initial states

In [6]:
num_tests = 100

# Define a dictionary to collect data for each algorithm
algorithm_data = {
    'Astar search': [],
    'Breadth First Tree Search': [],
    'Depth First Tree Search': [],
    'Depth Limited Search': [],
}

for _ in range(num_tests):
    results = run_singleTest(initial_state)

    for result in results:
        algorithm = result['Algorithm']  # Algorithm name
        success = result['Success'] # Success flag
        execution_time = round(result['Time'], 2)  # Execution time

        # Collect data for each algorithm
        algorithm_data[algorithm].append({
            'success': success,
            'execution_time': execution_time,
        })

# Analyze the collected data
for algorithm, data in algorithm_data.items():
    success_rate = sum(1 for entry in data if entry['success']) / num_tests
    avg_execution_time = sum(entry['execution_time'] for entry in data) / num_tests

    print(f"Algorithm: {algorithm}")
    print(f"Success Rate: {success_rate:.2%}")
    print(f"Average Execution Time: {avg_execution_time:.2f} seconds")
    print()



Algorithm: Astar search
Success Rate: 100.00%
Average Execution Time: 0.19 seconds

Algorithm: Breadth First Tree Search
Success Rate: 0.00%
Average Execution Time: 64.13 seconds

Algorithm: Depth First Tree Search
Success Rate: 0.00%
Average Execution Time: 60.73 seconds

Algorithm: Depth Limited Search
Success Rate: 0.00%
Average Execution Time: 23.14 seconds

